In [1]:
import sys
sys.path.append("..")
from damage_indicator_module import*

import copy
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
set_plot_formatting()

### INPUTS

In [2]:
# Building results file
building_id = ['ID1008','ID1012','ID1014','ID1021']
im = 'sa_avg'

# Damage indicator names
di_types =  ["$SDR_{peak}$", "$RSDR_{peak}$", "$FDI_{peak}$", "$FDI_{max}^{bottom}$", "Beams DS$\geq$1", \
             "$Sa(T_1)^{mainshock}/Sa(T_1)^{DBE}$"]
x_tick_names = ["$SDR_{peak}$", "$SDR_{residual}$", "$FDI_{max}$", "$FDI_{max}^{bottom}$", \
                "Beams\nDS$\geq$1", "$Sa(T)^{main}/$\n$Sa(T)^{DBE}$"]


# Minimum value of each damage indicator to consider
di_min = [1e-4, 1e-6, 1e-7, 1e-6, 1e-6, 1e-6, 1e-6, 1e-6, 1e-6]

# Transformation for fit and display
di_transformation = "log"

n_scales = 7
n_mainshocks = 44

#### Retrieve damage indicator data per buildng

In [3]:
damage_instances_data = dict()
for b_i in building_id:
    damage_instances_data[b_i] = pd.read_hdf('Damage_Instances_per_building.h5', key=b_i)

#### Assemble Damage Indicator Matrix per building [check manually those related to fractions of damaged components]

In [4]:
damage_instances_matrix = dict()

for b_i in building_id:
    # Retrieve data of current building
    current_building = damage_instances_data[b_i]
    
    # Matrix of Damage Indicators per damage instance        
    di_matrix = current_building[di_types[0]]
    di_matrix = np.vstack((di_matrix, current_building[di_types[1]]))
    di_matrix = np.vstack((di_matrix, current_building[di_types[2]]))
    di_matrix = np.vstack((di_matrix, current_building[di_types[3]]))
    di_matrix = np.vstack((di_matrix, 1 - np.stack(current_building['dsr_beams'].to_numpy())[:,0]))
    di_matrix = np.vstack((di_matrix, current_building[di_types[5]]))
    
    damage_instances_matrix[b_i] = copy.deepcopy(np.transpose(di_matrix))
del(di_matrix)

#### Reduction in median collapse capacity

In [5]:
k_min = 0.05

k = dict()

for b_i in building_id:
    # Retrieve data of current building
    k[b_i] = damage_instances_data[b_i]['$\kappa$'].values    
    damage_instances_matrix[b_i] = damage_instances_matrix[b_i][k[b_i] > k_min]
    damage_instances_data[b_i] = damage_instances_data[b_i][k[b_i] > k_min]
    k[b_i] = k[b_i][k[b_i] > k_min]

y_label = '$\kappa$' #'Fraction of $\widehat{Sa}_{collapse}^{Intact}$, $\kappa$'


### TRAIN TRILINEAR MODEL

In [6]:
parameters = dict()
N_seeds = 20

for b_i in building_id:
    parameters_bi = pd.DataFrame()
    
    for j, di in enumerate(di_types):        
        temp = fitPieceWiseFunc3LinLS(di, damage_instances_matrix[b_i][:, j], k[b_i], di_min[j], di_transformation, N_seeds)
    
        parameters_bi = parameters_bi.append(temp)
        print(b_i + ' ' + di)
        
    parameters[b_i] = copy.deepcopy(parameters_bi)
    parameters[b_i]
    
del(parameters_bi)


..\damage_indicator_module\damage_indicator_module\damage_indicators.py:669: RuntimeWarning: invalid value encountered in double_scalars
  b1_0 = np.sum(x1.dot(y1)) / np.sum(x1.dot(x1))
..\damage_indicator_module\damage_indicator_module\damage_indicators.py:585: RuntimeWarning: invalid value encountered in greater
  y_seg1 = y[np.logical_and(x > param[1], x <= param[2])]
..\damage_indicator_module\damage_indicator_module\damage_indicators.py:585: RuntimeWarning: invalid value encountered in less_equal
  y_seg1 = y[np.logical_and(x > param[1], x <= param[2])]
..\damage_indicator_module\damage_indicator_module\damage_indicators.py:586: RuntimeWarning: invalid value encountered in greater
  y_seg2 = y[x > param[2]]
..\damage_indicator_module\damage_indicator_module\damage_indicators.py:588: RuntimeWarning: invalid value encountered in greater
  y_pred_seg1 = y_pred[np.logical_and(x > param[1], x <= param[2])]
..\damage_indicator_module\damage_indicator_module\damage_indicators.py:588: Run

ID1008 $SDR_{peak}$
ID1008 $RSDR_{peak}$
ID1008 $FDI_{peak}$
ID1008 $FDI_{max}^{bottom}$
ID1008 Beams DS$\geq$1
ID1008 $Sa(T_1)^{mainshock}/Sa(T_1)^{DBE}$


,Minimum limit,Threshold limit,Slope 1,Slope 2,k_max,std_residuals,Residuals,x_min,di for plot,k for plot
$SDR_{peak}$,-5.179026,-3.037432,-0.122262,-0.511826,0.990306,0.121504,"[0.02422509909898707, 0.0940155668403776, 0.12...",1.000000e-04,"[0.004644399999999998, 0.0056334919234966845, ...","[0.9903057477039257, 0.9903057477039257, 0.728..."
$RSDR_{peak}$,-7.969114,-3.403685,-0.048779,-0.432499,0.958400,0.114019,"[0.005745865436240005, 0.09187943664439535, 0....",1.000000e-06,"[1.64539e-05, 0.00034598528412347045, 0.033250...","[0.9583995670743236, 0.9583995670743236, 0.735..."
$FDI_{peak}$,-5.684488,-1.141647,-0.034222,-0.523584,0.959557,0.105225,"[0.010336046297249935, 0.05542115941077219, 0....",1.000000e-07,"[2.2639182824721564e-07, 0.003398273863180998,...","[0.9595566423958053, 0.9595566423958053, 0.804..."
$FDI_{max}^{bottom}$,-7.007530,-1.265399,-0.027128,-0.430206,0.969055,0.106435,"[0.022205097266884022, 0.052383524767197764, 0...",1.000000e-06,"[1.0000000000000004e-06, 0.0009050412557893872...","[0.9690554541273856, 0.9690554541273856, 0.813..."
Beams DS$\geq$1,-9.860556,-1.541706,-0.013983,-0.196235,0.989563,0.128211,"[0.004555584431171433, 0.14401106085818416, 0....",1.000000e-06,"[1.0000000000000004e-06, 5.219332852404978e-05...","[0.9895631410191065, 0.9895631410191065, 0.873..."
$Sa(T_1)^{mainshock}/Sa(T_1)^{DBE}$,-0.702584,0.728571,-0.185311,-0.063573,0.974175,0.138754,"[0.07805604198245653, 0.16939568652442727, 0.1...",1.000000e-06,"[0.2818333333333334, 0.4953037482870381, 2.072...","[0.9741748599312186, 0.9741748599312186, 0.708..."


ID1012 $SDR_{peak}$
ID1012 $RSDR_{peak}$
ID1012 $FDI_{peak}$
ID1012 $FDI_{max}^{bottom}$
ID1012 Beams DS$\geq$1
ID1012 $Sa(T_1)^{mainshock}/Sa(T_1)^{DBE}$


,Minimum limit,Threshold limit,Slope 1,Slope 2,k_max,std_residuals,Residuals,x_min,di for plot,k for plot
$SDR_{peak}$,-4.616424,-3.691101,-0.095888,-0.338293,1.014963,0.084902,"[0.04060408541444249, 0.07785639374225939, 0.1...",1.000000e-04,"[0.003451720000000001, 0.009888090266025771, 0...","[1.0149629235067872, 1.0149629235067872, 0.926..."
$RSDR_{peak}$,-6.770477,-4.049708,-0.054174,-0.279954,1.005146,0.086225,"[0.05042079016243051, 0.06962133518792013, 0.1...",1.000000e-06,"[1.4141999999999996e-05, 0.0011471472976703175...","[1.0051462187587992, 1.0051462187587992, 0.857..."
$FDI_{peak}$,-4.911872,-1.762935,-0.034315,-0.288677,1.013494,0.079423,"[0.042072750888059396, 0.0845911435632688, 0.0...",1.000000e-07,"[9.999999999999994e-08, 0.0073587012874460865,...","[1.0134942580331703, 1.0134942580331703, 0.905..."
$FDI_{max}^{bottom}$,-5.217994,-1.765176,-0.032618,-0.296677,1.015960,0.077659,"[0.03960722087219648, 0.06023819096214433, 0.0...",1.000000e-06,"[1.0000000000000004e-06, 0.005418188110632069,...","[1.0159597880490332, 1.0159597880490332, 0.903..."
Beams DS$\geq$1,-6.389491,-2.019680,-0.016261,-0.171536,1.016967,0.106796,"[0.03859965945237942, 0.16625475100109477, 0.1...",1.000000e-06,"[1.0000000000000004e-06, 0.0016791100989700567...","[1.0169673494688503, 1.0169673494688503, 0.945..."
$Sa(T_1)^{mainshock}/Sa(T_1)^{DBE}$,-0.495122,0.048753,-0.248784,-0.094701,1.003632,0.119714,"[0.1245249847130303, 0.1805090942354597, 0.179...",1.000000e-06,"[0.21599999999999997, 0.6094966699583027, 1.04...","[1.0036323959342826, 1.0036323959342826, 0.868..."


ID1014 $SDR_{peak}$
ID1014 $RSDR_{peak}$
ID1014 $FDI_{peak}$
ID1014 $FDI_{max}^{bottom}$
ID1014 Beams DS$\geq$1
ID1014 $Sa(T_1)^{mainshock}/Sa(T_1)^{DBE}$


,Minimum limit,Threshold limit,Slope 1,Slope 2,k_max,std_residuals,Residuals,x_min,di for plot,k for plot
$SDR_{peak}$,-4.485729,-3.573504,-0.177548,-0.489646,1.002444,0.075832,"[0.07518173295775077, 0.08383380040408595, 0.1...",1.000000e-04,"[0.0026110000000000005, 0.011268668254924879, ...","[1.0024441818325995, 1.0024441818325995, 0.840..."
$RSDR_{peak}$,-5.463609,-2.494858,-0.165030,-11.535306,0.961240,0.086615,"[0.104308163269617, 0.08150000693087955, 0.156...",1.000000e-06,"[4.213559999999999e-05, 0.004238231779138081, ...","[0.9612399631257222, 0.9612399631257222, 0.471..."
$FDI_{peak}$,-4.935748,-2.060617,-0.037618,-0.277885,1.000732,0.079429,"[0.06811968427378934, 0.07056181135487605, 0.0...",1.000000e-07,"[9.999999999999994e-08, 0.007185087097523824, ...","[1.0007321190975504, 1.0007321190975504, 0.892..."
$FDI_{max}^{bottom}$,-5.704240,-2.055049,-0.030628,-0.307640,1.005193,0.076388,"[0.060354932455652044, 0.07099506507482667, 0....",1.000000e-06,"[1.0000000000000004e-06, 0.0033318098703696234...","[1.0051931939396872, 1.0051931939396872, 0.893..."
Beams DS$\geq$1,-5.369958,-2.069206,-0.026982,-0.254921,1.002406,0.099179,"[0.0841508704418148, 0.0949675818667245, 0.187...",1.000000e-06,"[1.0000000000000004e-06, 0.004654328674675539,...","[1.0024056627675653, 1.0024056627675653, 0.913..."
$Sa(T_1)^{mainshock}/Sa(T_1)^{DBE}$,-0.729255,1.072951,-0.176954,-0.604651,1.006718,0.119522,"[0.09163520097371125, 0.11971724982324383, 0.1...",1.000000e-06,"[0.15766666666666668, 0.48226819271497745, 2.9...","[1.0067184006590435, 1.0067184006590435, 0.687..."


ID1021 $SDR_{peak}$
ID1021 $RSDR_{peak}$
ID1021 $FDI_{peak}$
ID1021 $FDI_{max}^{bottom}$
ID1021 Beams DS$\geq$1
ID1021 $Sa(T_1)^{mainshock}/Sa(T_1)^{DBE}$


,Minimum limit,Threshold limit,Slope 1,Slope 2,k_max,std_residuals,Residuals,x_min,di for plot,k for plot
$SDR_{peak}$,-5.005123,-3.771173,-0.121157,-0.366578,0.981040,0.095356,"[0.03496562307779072, 0.05982821305317676, 0.0...",1.000000e-04,"[0.00164587, 0.0067035149941669665, 0.02302504...","[0.9810398903330609, 0.9810398903330609, 0.831..."
$RSDR_{peak}$,-8.247988,-4.328894,-0.051187,-0.244900,0.996145,0.090176,"[0.09313214464582154, 0.08623544453826337, 0.0...",1.000000e-06,"[1.229099999999999e-05, 0.0002617848556086024,...","[0.9961448007182824, 0.9961448007182824, 0.795..."
$FDI_{peak}$,-6.407768,-1.789179,-0.027320,-0.334115,0.978429,0.090205,"[0.03906358576685687, 0.05290632455138655, 0.0...",1.000000e-07,"[9.999999999999994e-08, 0.0016486998477033513,...","[0.9784293433724217, 0.9784293433724217, 0.852..."
$FDI_{max}^{bottom}$,-6.686847,-1.787124,-0.026073,-0.335778,0.980072,0.089746,"[0.042605734749470936, 0.05317980052177629, 0....",1.000000e-06,"[1.0000000000000004e-06, 0.0012472093799246596...","[0.9800722358469043, 0.9800722358469043, 0.852..."
Beams DS$\geq$1,-7.343397,-1.894302,-0.022583,-0.396836,0.981494,0.110006,"[0.04407933719687629, 0.05451051365037185, 0.1...",1.000000e-06,"[1.0000000000000004e-06, 0.0006468494221177127...","[0.9814942915422729, 0.9814942915422729, 0.858..."
$Sa(T_1)^{mainshock}/Sa(T_1)^{DBE}$,-0.798970,1.462731,-0.167847,-1.477515,0.990809,0.124669,"[0.09300038526768928, 0.15156640736919424, 0.1...",1.000000e-06,"[0.20366666666666672, 0.44979205775243297, 4.3...","[0.9908085132436443, 0.9908085132436443, 0.611..."


### SAVE REGRESSION MODEL PARAMETERS

In [7]:
for b_i in building_id:
    parameters_bi = parameters[b_i]
    parameters_bi.to_hdf('kappa_function_parameters.h5', key=b_i)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['Residuals', 'di for plot', 'k for plot']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
